<h3 style="color: #000000; font-weight: bold;"> Gen AI Project: Building a Finantial Bot </h3>

##### **Goal of the Project** : The objective of the project is to deploy a chatbot capable of providing financial advice based on the book **_Rich Dad Poor Dad_** by **Robert T. Kiyosaki**.


In [1]:
# importing the libraries

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from chromadb.config import Settings
import chromadb
import openai

<h3 style="color: #000000; font-weight: bold;"> ETL </h3>

In [2]:
# PDF location for retrieving the data
document_dir = "./"
filename = "Rich Dad Poor Dad.pdf"
file_path = os.path.join(document_dir, filename)

In [3]:
# Load and split the document
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)

239

In [4]:
# Split pages into chunks
text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=0) # CharacterTextSplitter - 
chunks = text_splitter.split_documents(pages)
len(chunks)

239

In [5]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [6]:
client = chromadb.PersistentClient(path="./chroma_db") # initializes the new environment , what kind of client

In [7]:
db = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db", client=client)

<h3 style="color: #000000; font-weight: bold;"> System Architecture </h3>

In [8]:
user_question = " I finished Ironhack Bootcamp and I invest on my education. How can I earn the money I spent?" # User question stored in a variable
retrieved_docs = db.similarity_search(user_question, k=5) # k is the number of documents to retrieve the context

In [9]:
# Display top results
for i, doc in enumerate(retrieved_docs[:1]): # Display top 3 results
    print(f"Document {i+1}:\n{doc.page_content[36:1000]}") # Display content 

Document 1:
ately $190,000
was created in the asset column, and no taxes were paid.
Which one sounds harder to you?
1. Work hard. Pay 50% in taxes. Save what is
left.
Your savings then earn 5%, which is also
taxed.
OR
2. Take the time to develop your ﬁnancial
intelligence
Harness the power of your brain and the
asset column.
If you use option number one, be sure to factor in how
much time it takes you to save $190,000. Time is one of
your greatest assets.
Now you may understand why I silently shake my head
when I hear parents say, “My child is doing well in school
and receiving a good education.” It may be good, but is it
adequate?
I know the above investment strategy is a small one. It is
used to illustrate how small can grow into big. Again, my
success reﬂects the importance of a strong ﬁnancial
foundation, which starts with a strong ﬁnancial education.
I have said it before, but it’s worth repeating. Financial
intelligence is made up of these four main techni


<h3 style="color: #000000; font-weight: bold;"> Deployment & System Integration </h3>

In [10]:
def _get_document_prompt(docs):
    prompt = "\n"
    for doc in docs:
        prompt += "\nContent:\n"
        prompt += doc.page_content + "\n\n"
    return prompt

In [11]:
# Generate a formatted context from the retrieved documents
formatted_context = _get_document_prompt(retrieved_docs)
print("Context formatted for GPT model.{}".format(formatted_context))

Context formatted for GPT model.

Content:
For about 30 hours of work, approximately $190,000
was created in the asset column, and no taxes were paid.
Which one sounds harder to you?
1. Work hard. Pay 50% in taxes. Save what is
left.
Your savings then earn 5%, which is also
taxed.
OR
2. Take the time to develop your ﬁnancial
intelligence
Harness the power of your brain and the
asset column.
If you use option number one, be sure to factor in how
much time it takes you to save $190,000. Time is one of
your greatest assets.
Now you may understand why I silently shake my head
when I hear parents say, “My child is doing well in school
and receiving a good education.” It may be good, but is it
adequate?
I know the above investment strategy is a small one. It is
used to illustrate how small can grow into big. Again, my
success reﬂects the importance of a strong ﬁnancial
foundation, which starts with a strong ﬁnancial education.
I have said it before, but it’s worth repeating. Financial
intell

<h3 style="color: #000000; font-weight: bold;"> NLP & Promp engineering </h3>

In [16]:
prompt = f"""
## SYSTEM ROLE
You are a highly knowledgeable and creative chatbot designed to assist with financial advice based on the principles outlined in **Rich Dad Poor Dad**.  
Your responses must be rooted exclusively in the provided content, focusing on explaining, comparing, and contextualizing key concepts.

## USER QUESTION
The user has asked:  
**"{user_question}"**

## CONTEXT
Here is the relevant content from the technical books:  
'''
{formatted_context}
'''

## GUIDELINES
1. **Accuracy**:  
   - Use only the information provided in the `CONTEXT` section to answer the question.  
   - If the context does not contain relevant information, clearly state: "The provided context does not contain this information."  
   - Begin by explaining the significance of the user's question.  
   - Follow up with a structured explanation, including:  
     - The perspective of **Rich Dad**.  
     - The perspective of **Poor Dad**.  
     - A comparison of both approaches.  
   - Conclude by highlighting why the **Rich Dad** approach is advantageous.  
   - Use analogies to engage the user and deepen their understanding.

2. **Formatting**:  
   - Keep the response concise.  
   - If the answer is too long, split it into shorter sentences and write the continuation on the next line.  
   - Highlight **Rich Dad** and **Poor Dad** in bold for clarity.  
   - Ensure the user's question appears in bold to differentiate it from the answer.

3. **Transparency**:  
   - Reference the book title whenever possible to provide credibility to the response.  
   - Avoid adding opinions or speculation outside the given context.

4. **Clarity**:  
   - Use concise, professional, and user-friendly language.  
   - Format the response in Markdown for enhanced readability.  

## TASK
1. Provide a direct and comprehensive answer to the user's question.  
2. Incorporate the **Rich Dad** and **Poor Dad** perspectives into your explanation.  
3. Use an analogy to simplify complex concepts and make the response more relatable.  
4. Deliver the response in the following format:

## RESPONSE FORMAT
'''
# [Title Reflecting the Answer]
**User Question:**  
**[The user's question.]**

**Answer:**  
[Provide a clear and concise answer to the question. If the answer is too long, continue on the next line.]

**Comparison:**  
- **Rich Dad Approach**: [Explanation of this approach. Keep it concise.]  
  [Continue explanation if needed.]  
- **Poor Dad Approach**: [Explanation of this approach. Keep it concise.]  
  [Continue explanation if needed.]  
- **Why Rich Dad's Approach is Better**: [Reasoning. Keep it concise.]  
  [Continue explanation if needed.]

**Analogy:**  
[Provide a simple and engaging analogy to support the explanation.]

**Source**:  
• [Book Title], Page(s): [...]
'''
"""
print("Prompt constructed.")


Prompt constructed.


<h3 style="color: #000000; font-weight: bold;"> Maintenance & Improvements </h3>

In [13]:
# Set up GPT client and parameters
client = openai.OpenAI()
model_params = {
    'model': 'gpt-4o',
    'temperature': 0.7,  # Increase creativity
    'max_tokens': 4000,  # Allow for longer responses
    'top_p': 0.9,        # Use nucleus sampling
    'frequency_penalty': 0.5,  # Reduce repetition
    'presence_penalty': 0.6    # Encourage new topics
}

In [14]:
messages = [{'role': 'user', 'content': prompt}]
completion = client.chat.completions.create(messages=messages, **model_params, timeout=120)

In [15]:
answer = completion.choices[0].message.content
print(answer)

'''
# Turning Education into Wealth

**User Question:**  
**"I finished Ironhack Bootcamp and I invest on my education. How can I earn the money I spent?"**

**Answer:**  
To earn back the money you invested in your education, focus on developing financial intelligence and leveraging your skills to create assets that generate income. Rather than just working a job, consider how you can apply what you've learned to build wealth.

**Comparison:**  
- **Rich Dad Approach**: Emphasizes building financial intelligence by investing time in learning about assets and creating them. This could mean starting a business or investing in real estate or other ventures that grow your wealth over time without being heavily taxed.
- **Poor Dad Approach**: Focuses on traditional employment and saving what's left after taxes. This approach limits potential growth since savings are typically subject to lower interest rates and taxes.
- **Why Rich Dad's Approach is Better**: The emphasis is on using financ